In [1]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.svm import LinearSVC
import numpy as np
import scipy

In [2]:
## nonlinearities
def relu(x, thrsh=0):
    return np.maximum(x, thrsh)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
class RFClassifier(BaseEstimator, ClassifierMixin):
    """
    Random features with SVM classification
    """
    
    def __init__(self, width=500, weights='white noise', nonlinearity=relu, clf=None, weight_fun=None, clf_args={}, 
                 seed=20):
        self.width=width
        self.weights = weights
        self.nonlinearity = nonlinearity
        self.clf = clf
        self.weight_fun = weight_fun
        self.clf_args = clf_args
        self.seed=seed

    def fit(self, X, y):
        # check params
        if X.shape[0] != y.shape[0]:
            raise ValueError('dimension mismatch')
        n = X.shape[1]

        if self.clf is None:
            self.clf = LinearSVC(random_state=self.seed, tol=1e-4, max_iter=1000)
        else:
            self.clf = self.clf(**self.clf_args)
        
        if self.weight_fun is not None:
            self.W_ = self.weight_fun(self.width, n)
        else:
            self.W_ = \
            random_feature_matrix(self.width, n, self.weights, self.seed)
        H = self.nonlinearity(X @ self.W_)
        
        #fit classifier
        self.clf.fit(H, y)
        self._fitted = True
        return self

    def transform(self, X):
        H = self.nonlinearity(X @ self.W_)
        return H
    
    def predict(self, X):
        H = self.nonlinearity(X @ self.W_)
        return self.clf.predict(H)
    
    def score(self, X, y):
        H = self.nonlinearity(X @ self.W_)
        return self.clf.score(H, y)

In [10]:
def gaussian(X, mu, sigma):
    return np.exp(-np.abs(mu - X) ** 2/ (2 * sigma **2))

In [11]:
def random_feature_matrix(M, N, weights='white noise', rand_seed=20):
    ''' 
    Generate a size (M, N) random matrix from the specified distribution
    
    Parameters
    ----------
    
    M: number of rows
    
    N: number of columns
    
    weights: string or function, default 'gaussian.'
    If 'unimodal', entries are gaussians with means ~ Unif(0, 1), and std. dev ~ Unif(0.1, N).
    If 'white noise', entries are drawn ~ N(0, 1).
    Or can be a function handle taking arguments (M, N)
    '''
    
    from numpy.random import randn, uniform, seed
    
    seed(rand_seed)
    if weights == 'unimodal':
        mu = uniform(0, N, (M, 1))
        sigma = uniform(0.1, N/4, (M, 1))
        k = np.arange(0, N)
        J = np.array([gaussian(k, m, s) for (m, s) in zip(mu, sigma)]) * np.random.randint(1, 20, (M, 1))

    elif weights == 'white noise':
        J = randn(M, N)
    
    else:
        J = weights(M, N)
    return J.T

In [13]:
# tests to check the estimator.py
from data_fns import noisy_signal, data_matrix, butter_bandpass_filter

f_s = 2000
dur = 10
f_sig = 40
sig_dur = 0.05

# neuron memory
delta = 0.08
N = int(delta * f_s)

signal, label = noisy_signal(f_s=f_s, dur=dur, f_signal=f_sig, sig_dur=sig_dur)
X, y = data_matrix(signal, label, N)
clf = RFClassifier(width=30, weights='white noise', weight_fun=None)
clf.fit(X, y)
clf.score(X, y)

/usr/local/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


0.5379032258064517

20